<a href="https://colab.research.google.com/github/jeonggunlee/CUDATeaching/blob/master/02_cuda_lab/00_googleDrive_CUDAExam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Google Drive를 활용한 CUDA 프로그래밍

직접 Google Drive에서 프로그램을 수행할 수 있습니다. 

이경우 mount가 필요하고 **여러개**의 화일로 이루어진 코드들을 수행할 수 있습니다.

또한 직접 구글 드라이브로 **git clone** 하여 소스를 살펴보고 실 데이터와 함께 수행 시킬 수 있습니다.

In [0]:
from google.colab import drive

Enter your authorization code:

지정된 URL을 클릭하면 인증 코드를 볼수 있으며, 이를 복사하여 입력합니다.

In [2]:
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [3]:
%cd /content

/content


In [4]:
!ls

gdrive	sample_data


In [5]:
%cd gdrive

/content/gdrive


In [6]:
!ls

'My Drive'


In [7]:
%cd My Drive

/content/gdrive/My Drive


In [8]:
%cd GPUCoding

/content/gdrive/My Drive/GPUCoding


In [9]:
!ls 

cs344


In [0]:
!git clone https://github.com/jeonggunlee/cs344/

Cloning into 'cs344'...
remote: Enumerating objects: 726, done.
remote: Total 726 (delta 0), reused 0 (delta 0), pack-reused 726
Receiving objects: 100% (726/726), 66.62 MiB | 21.37 MiB/s, done.
Resolving deltas: 100% (402/402), done.
Checking out files: 100% (135/135), done.


In [10]:
!ls

cs344


In [11]:
%cd cs344

/content/gdrive/My Drive/GPUCoding/cs344


In [12]:
!ls

 CMakeLists.txt		'Lesson Slides'  'Student Contributions'
 Final			'Problem Sets'
'Lesson Code Snippets'	 README.md


In [13]:
%cd 'Lesson Code Snippets'

/content/gdrive/My Drive/GPUCoding/cs344/Lesson Code Snippets


In [14]:
!ls

'Lesson 2 Code Snippets'  'Lesson 5 Code Snippets'
'Lesson 3 Code Snippets'  'Lesson 7 Code Snippets'


In [15]:
%cd 'Lesson 2 Code Snippets'

/content/gdrive/My Drive/GPUCoding/cs344/Lesson Code Snippets/Lesson 2 Code Snippets


In [16]:
!ls

associative	atomics     gputimer.h	       hello_threadIdx	   memory
associative.cu	atomics.cu  hello_blockIdx.cu  hello_threadIdx.cu  memory.cu


In [0]:
!nvcc -o hello_threadIdx hello_threadIdx.cu

In [18]:
!./hello_threadIdx

Hello world! I'm thread 0
Hello world! I'm thread 1
Hello world! I'm thread 2
Hello world! I'm thread 3
That's all!


In [19]:
!ls

associative	atomics     gputimer.h	       hello_threadIdx	   memory
associative.cu	atomics.cu  hello_blockIdx.cu  hello_threadIdx.cu  memory.cu


In [20]:
!cat hello_threadIdx.cu

#include <stdio.h>

#define NUM_BLOCKS 1
#define BLOCK_WIDTH 4

__global__ void hello()
{
    printf("Hello world! I'm thread %d\n", threadIdx.x);
}


int main(int argc,char **argv)
{
    // launch the kernel
    hello<<<NUM_BLOCKS, BLOCK_WIDTH>>>();

    // force the printf()s to flush
    cudaDeviceSynchronize();

    printf("That's all!\n");

    return 0;
}


### 위의 코드는 다음과 같습니다.

```c
#include <stdio.h>

// block의 수 정의
#define NUM_BLOCKS 1
// block 내부의 thread 수 정의
#define BLOCK_WIDTH 4

__global__ void hello()
{
    printf("Hello world! I'm thread %d\n", threadIdx.x);
}

int main(int argc,char **argv)
{
    // launch the kernel
    hello<<<NUM_BLOCKS, BLOCK_WIDTH>>>();

// force the printf()s to flush
    cudaDeviceSynchronize();

    printf("That's all!\n");
    return 0;
}

```

In [21]:
!ls

associative	atomics     gputimer.h	       hello_threadIdx	   memory
associative.cu	atomics.cu  hello_blockIdx.cu  hello_threadIdx.cu  memory.cu


In [22]:
!nvcc -o memory memory.cu

memory.cu(11): warning: variable "f" was set but never used



### memory.cu의 내부는 다음과 같습니다.

다양한 메모리 형태에 대한 사용방법을 내재하고 있습니다.

- local memory
- global memory
- shared memory


```c
// Using different memory spaces in CUDA
#include <stdio.h>

/**********************
 * using local memory *
 **********************/

// a __device__ or __global__ function runs on the GPU
__global__ void use_local_memory_GPU(float in)
{
    float f;    // variable "f" is in local memory and private to each thread
    f = in;     // parameter "in" is in local memory and private to each thread
    // ... real code would presumably do other stuff here ... 
}

/**********************
 * using global memory *
 **********************/

// a __global__ function runs on the GPU & can be called from host
__global__ void use_global_memory_GPU(float *array)
{
    // "array" is a pointer into global memory on the device
    array[threadIdx.x] = 2.0f * (float) threadIdx.x;
}

/**********************
 * using shared memory *
 **********************/

// (for clarity, hardcoding 128 threads/elements and omitting out-of-bounds checks)
__global__ void use_shared_memory_GPU(float *array)
{
    // local variables, private to each thread
    int i, index = threadIdx.x;
    float average, sum = 0.0f;

    // __shared__ variables are visible to all threads in the thread block
    // and have the same lifetime as the thread block
    __shared__ float sh_arr[128];

    // copy data from "array" in global memory to sh_arr in shared memory.
    // here, each thread is responsible for copying a single element.
    sh_arr[index] = array[index];

    __syncthreads();    // ensure all the writes to shared memory have completed

    // now, sh_arr is fully populated. Let's find the average of all previous elements
    for (i=0; i<index; i++) { sum += sh_arr[i]; }
    average = sum / (index + 1.0f);

    // if array[index] is greater than the average of array[0..index-1], replace with average.
    // since array[] is in global memory, this change will be seen by the host (and potentially 
    // other thread blocks, if any)
    if (array[index] > average) { array[index] = average; }

    // the following code has NO EFFECT: it modifies shared memory, but 
    // the resulting modified data is never copied back to global memory
    // and vanishes when the thread block completes
    sh_arr[index] = 3.14;
}

int main(int argc, char **argv)
{
    /*
     * First, call a kernel that shows using local memory 
     */
    use_local_memory_GPU<<<1, 128>>>(2.0f);

    /*
     * Next, call a kernel that shows using global memory
     */
    float h_arr[128];   // convention: h_ variables live on host
    float *d_arr;       // convention: d_ variables live on device (GPU global mem)

    // allocate global memory on the device, place result in "d_arr"
    cudaMalloc((void **) &d_arr, sizeof(float) * 128);
    // now copy data from host memory "h_arr" to device memory "d_arr"
    cudaMemcpy((void *)d_arr, (void *)h_arr, sizeof(float) * 128, cudaMemcpyHostToDevice);
    // launch the kernel (1 block of 128 threads)
    use_global_memory_GPU<<<1, 128>>>(d_arr);  // modifies the contents of array at d_arr
    // copy the modified array back to the host, overwriting contents of h_arr
    cudaMemcpy((void *)h_arr, (void *)d_arr, sizeof(float) * 128, cudaMemcpyDeviceToHost);
    // ... do other stuff ...

    /*
     * Next, call a kernel that shows using shared memory
     */

    // as before, pass in a pointer to data in global memory
    use_shared_memory_GPU<<<1, 128>>>(d_arr); 
    // copy the modified array back to the host
    cudaMemcpy((void *)h_arr, (void *)d_arr, sizeof(float) * 128, cudaMemcpyHostToDevice);
    // ... do other stuff ...
    return 0;
}
```

In [0]:
!./memory

In [28]:
!nvprof ./memory

==657== NVPROF is profiling process 657, command: ./memory
==657== Profiling application: ./memory
==657== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   38.87%  5.0870us         1  5.0870us  5.0870us  5.0870us  use_shared_memory_GPU(float*)
                   19.32%  2.5280us         1  2.5280us  2.5280us  2.5280us  use_global_memory_GPU(float*)
                   14.66%  1.9190us         1  1.9190us  1.9190us  1.9190us  use_local_memory_GPU(float)
                   14.43%  1.8880us         1  1.8880us  1.8880us  1.8880us  [CUDA memcpy HtoD]
                   12.72%  1.6640us         1  1.6640us  1.6640us  1.6640us  [CUDA memcpy DtoH]
      API calls:   99.70%  233.50ms         3  77.832ms  8.0780us  233.47ms  cudaLaunchKernel
                    0.13%  293.73us         1  293.73us  293.73us  293.73us  cudaMalloc
                    0.07%  175.00us         1  175.00us  175.00us  175.00us  cuDeviceTotalMem
      

In [29]:
!nvprof --print-gpu-trace ./memory

==682== NVPROF is profiling process 682, command: ./memory
==682== Profiling application: ./memory
==682== Profiling result:
   Start  Duration            Grid Size      Block Size     Regs*    SSMem*    DSMem*      Size  Throughput  SrcMemType  DstMemType           Device   Context    Stream  Name
375.71ms  1.8880us              (1 1 1)       (128 1 1)        16        0B        0B         -           -           -           -     Tesla T4 (0)         1         7  use_local_memory_GPU(float) [106]
376.04ms  1.8560us                    -               -         -         -         -      512B  263.08MB/s    Pageable      Device     Tesla T4 (0)         1         7  [CUDA memcpy HtoD]
376.06ms  2.4000us              (1 1 1)       (128 1 1)        16        0B        0B         -           -           -           -     Tesla T4 (0)         1         7  use_global_memory_GPU(float*) [109]
376.07ms  1.6320us                    -               -         -         -         -      512B  299.

In [0]:
!nvcc -o associative associative.cu

In [25]:
!./associative

(1 + 1e+99) + -1e+99 == 0
1 + (1e+99 + -1e+99) == 1


In [0]:
!nvcc -o atomics atomics.cu

In [27]:
!./atomics

1000000 total threads in 1000 blocks writing into 100 array elements
{ 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 }
Time elapsed = 0.18976 ms
